In [ ]:
using LilGuys
using CairoMakie
using Arya

In [ ]:
import TOML
using CSV, DataFrames

In [ ]:
using OrderedCollections

In [ ]:
function load_profile(galaxyname)
    prof = LilGuys.StellarProfile(joinpath(galaxyname, "density_profiles/jax_profile.toml"))
    # props = TOML.parsefile(joinpath(galaxyname, "observed_properties.toml"))

    # prof.log_r = prof.log_r .+ log10.(LilGuys.arcmin_to_kpc(1, props["distance"]))

    # if prof.normalization == "none"
    #     prof.log_Sigma .-= log10(prof.M_in[end])
    # end
    norm = LilGuys.lerp(prof.log_r, prof.log_Sigma)(0)
    prof.log_Sigma .-= norm
    return prof
end

In [ ]:
function load_props(galaxyname)
    props = TOML.parsefile(joinpath(galaxyname, "observed_properties.toml"))

    return props
end

In [ ]:
function load_gaia_obs(galaxyname)
    df = LilGuys.read_fits(joinpath(galaxyname, "data", "j24_2c.fits"))
    return df
end

In [ ]:
galaxies = [
    "aquarius2",
    "bootes1",
    "bootes2",
    "bootes3",
    "bootes4",
    "bootes5",
    "canesvenatici1",
    "canesvenatici2",
    "carina",
    "carina2",
    "carina3",
    "centaurus1",
    "cetus2",
    "columba1",
    "coma_berenices",
    "crater2",
    # "delve2",
    #DES
    "draco",
    "draco2",
    "eridanus2",
    # "eridanus3",
    "eridanus4",
    "fornax",
    # "grus1", fits err
    "grus2",
    #"hercules",
    "horologium1",
    "horologium2",
    "hydra2",
    "hydrus1",
    #"indus1",
    #"indus2",
    "leo1",
    "leo2",
    "leo4",
    "leo5",
     "leo_minor",
     "leo_t",
    # "pegasus3", no stars
     "pegasus4", 
    # "phoenix", fits err
     "phoenix2",
    "pictor2",
   # "pictoris1", obs props
    # "pisces2", 2 stars
    "reticulum2",
    "reticulum3",
    #"sagittarius2",
    "sculptor",
    "segue1",
    "segue2",
    "sextans1",
    "triangulum2",
    "tucana2",
    "tucana3",
    "tucana4",
    "tucana5",
    "unions1",
    "ursa_major1",
    "ursa_major2",
    "ursa_minor",
    "virgo1",
    # "virgo2" 2 stars,
    # "willman1",
]

In [ ]:
galaxies_flagged = [
    "bootes1",
    "bootes3",
    "draco2",
    "grus2",
    "sculptor",
    "segue1",
    "tucana2",
    "tucana3",
    "ursa_minor",
    ]

In [ ]:
profiles = OrderedDict(key => load_profile(key) for key in galaxies)

In [ ]:
properties = OrderedDict(key => load_props(key) for key in galaxies)

In [ ]:
for (label, prof) in profiles
    println(label)
    rh = LilGuys.quantile(prof.log_r, prof.counts, 0.5)

    println(rh, "\t", 10^rh)
    println("counts = ", sum(prof.counts))
    println("Mv = ", properties[label]["Mv"])
    println()
end

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "log r / r_h",
    ylabel=L"\log\,\Sigma",
   # limits=(-2, 2, -5, 5)
)

i = 0
colorrange=(-10, 0)
colormap = Reverse(Arya.get_arya_cmap())

for (label, prof) in profiles
    y = prof.log_Sigma

    if length(y) < 1
        continue
    end
    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=colorrange, colormap=colormap)
end


x = LinRange(-2, 1, 1000)
y = log10.(exponential_profile.(exp10.(x))) .- log10(exponential_profile(1))
lines!(x, y, color=COLORS[1])
Colorbar(fig[1,2], colorrange=colorrange, label="Mv (mag)", colormap=colormap)
fig

In [ ]:
function exponential_profile(x)
    r_s = 1/0.6931471805599453
    return 1/r_s * exp(-x / r_s)
end

In [ ]:
using QuadGK

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel = "log r / r_h", ylabel=L"\log\,\Sigma")
i = 0

for (label, prof) in profiles
    y = prof.log_Sigma
    if sum(prof.counts) < 30
        continue
    end

    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=colorrange, colormap=colormap)
end

x = LinRange(-2, 1, 1000)
y = log10.(exponential_profile.(exp10.(x))) .- log10(exponential_profile(1))
lines!(x, y, color=COLORS[1])

Colorbar(fig[1,2], colorrange=colorrange, label="Mv (mag)", colormap=colormap)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "log r / r_h", 
    ylabel=L"\log\,\Sigma",
    title="dwarfs with features",
)
i = 0

for (label, prof) in profiles
    y = prof.log_Sigma
    if label ∉ galaxies_flagged
        continue
    end

    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=colorrange, colormap=colormap)
end


Colorbar(fig[1,2], colorrange=colorrange, label="Mv (mag)", colormap=colormap)

x = LinRange(-2, 1, 1000)
y = log10.(exponential_profile.(exp10.(x))) .- log10(exponential_profile(1))
lines!(x, y, color=COLORS[1])


fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "log r / r_h", 
    ylabel=L"\log\,\Sigma",
    title="dwarfs without",
)
i = 0

for (label, prof) in profiles
    y = prof.log_Sigma
    if label ∈ galaxies_flagged || (sum(prof.counts) < 30)
        continue
    end

    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=(-10, 0))
end

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel = "log r / kpc", ylabel=L"\log\,\Sigma")
i = 0

dy = 1.0
for (label, prof) in profiles
    y = prof.log_Sigma .+ i*dy
    errscatter!(prof.log_r, y, yerr=prof.log_Sigma_err)
    lines!(prof.log_r, y)

    text!(-3, -2+i*0.8dy, text=label, color=COLORS[(i) % 7+1])
    i += 1
end

fig

In [ ]:
function scatter_obs(df)
    fig = Figure()
    ax = Axis(fig[1,1],
        aspect=DataAspect())

    scatter!(df.xi, df.eta, markersize=1, alpha=0.1, color=:black)
    scatter!(df.xi[df.PSAT .> 0.2], df.eta[df.PSAT .> 0.2], markersize=3, color=COLORS[2])

    LilGuys.hide_grid!(ax)
    fig
end

In [ ]:
df = load_gaia_obs("bootes1")

In [ ]:
scatter_obs(df)

In [ ]:
df = load_gaia_obs("segue1")

In [ ]:
scatter_obs(df)

In [ ]:
df = load_gaia_obs("draco2")

In [ ]:
scatter_obs(df)

In [ ]:
df = load_gaia_obs("grus2")

In [ ]:
scatter_obs(df)